In [1]:
import os
os.chdir("/nas/longleaf/home/cctsai/proj_dip/")
# /work/users/c/c/cctsai/data/BCP_sample/T2/

In [2]:
!jupyter nbconvert --to script submission_ablation.ipynb

exp_namespace = 'T2/KC-DIP'
submission_py = "submission_ablation.py"
%run "submission_ablation.ipynb"


# # # # # # 
"""

vvvvvvvvvvvvv
v) DIP+TV
v) dDIP, dDIP_noKspace (1.25)
v) sDIP, sDIP_noKspace (1.25, 1.6, 2)
v) without_kbound 但有kspace
v) kspace_mse_shape_without_boundary 
v) kspace_mse_shape
"""

"/nas/longleaf/home/cctsai"
"/work/users/c/c/cctsai"

[NbConvertApp] Converting notebook submission_ablation.ipynb to script
[NbConvertApp] Writing 12731 bytes to submission_ablation.py


usage: ipykernel_launcher.py [-h] [--file FILE] [--factor FACTOR]
                             [--model_folder MODEL_FOLDER] [--input_img]
                             [--double_arm DOUBLE_ARM] [--SSIM] [--kspace_mse]
                             [--kspace_mse_shape KSPACE_MSE_SHAPE]
                             [--kspace_mse_weight KSPACE_MSE_WEIGHT]
                             [--kspace_boundary]
                             [--kbound_weight KBOUND_WEIGHT]
                             [--kbound_lower KBOUND_LOWER]
                             [--kbound_outer_layer KBOUND_OUTER_LAYER]
ipykernel_launcher.py: error: unrecognized arguments: -f /nas/longleaf/home/cctsai/.local/share/jupyter/runtime/kernel-ac38e1cd-b2dd-4d9b-85d6-d42f6eb01f0d.json


SystemExit: 2

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


SystemExit: 2

In [3]:
# You can change the names here
#exp_namespace = 'ablation/kspace_boundary/noMSE/{}'

### fixed ###
template = """#!/bin/bash

#SBATCH --job-name=pytorch
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=50G
#SBATCH --time=40:00:00
#SBATCH --partition=a100-gpu
#SBATCH --output=slurm_log/{}/run-%j.log --open-mode=append
#SBATCH --gres=gpu:1
#SBATCH --qos=gpu_access

# Path to the package directory
PACKAGE_PATH=/nas/longleaf/home/cctsai/.local/lib/python3.9/site-packages

# Add the package path to Python's sys.path
export PYTHONPATH="$PYTHONPATH:$PACKAGE_PATH"

module load cuda/11.8
module load python/3.9.6

cd /nas/longleaf/home/cctsai/proj_dip/

python {} {}
"""

In [4]:
# ages = ['17', '51', '54', '92', '109', '146', '179', '205', '233', '272', '317', '342', '357']
# ages = ['17', '179', '272', '317', '357']
# ages = ['51', '179', '357']
ages = ['233', '342', '205', '146', '317', '92', '17', '54', '109', '272']


import glob, os
files = []

for age in ages:
    files += glob.glob(f"/work/users/c/c/cctsai/data/BCP_sample/T2/*_{age}_*.npy")
print(files)

def create_slurm_folders(task_name):
    for dst_dir in ['slurm', 'slurm_log']:
        new_folder = f"/nas/longleaf/home/cctsai/proj_dip/{dst_dir}/{task_name}"
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
            print('Create:',new_folder)

['/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP223008_233_T2w_SPC_NORM_5.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP522586_342_T2w_SPC_NORM_5.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP116056_205_T2w_SPC_NORM_9.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP439083_146_T2w_SPC_NORM_5.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP477045_317_T2w_SPC_NORM_5.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP206192_92_T2w_SPC_NORM_7.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/NCBCP062519_17_T2w_SPC_NORM_5.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP439083_54_T2w_SPC_NORM_5.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP116845_109_T2w_SPC_NORM_6.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP229768_272_T2w_SPC_NORM_5.npy']


In [5]:
# for factor in [2]:
#     for kbound_weight in [0.001, 0.0005, 0.0001]:
#         for kbound_outer in ['True','False']:
#             exp_name = exp_namespace.format(f'kbound_weight_{kbound_weight}/kbound_outer_{kbound_outer}')

factor = 2

# for kbound in ['kbound', 'nokbound']:
exp_name = f"{exp_namespace}"

###
task_name = exp_name
create_slurm_folders(task_name)
for npy_file in files:
    arg1 = f"--file {npy_file} --factor {factor} --model_folder {exp_name} --double_arm True "
#     arg2 = f"--double_arm False"
#     if kbound=="kbound":
    arg2 = f"--kspace_mse --kspace_boundary --kbound_outer_layer False"         
#     elif kbound=="nokbound":
#         arg2 = f"--kspace_mse --kbound_outer_layer False"
#     else:
#         raise ValueError("error")
    arg_strings = arg1 + arg2
    print(arg_strings)

    ### write to file
    script_content = template.format(task_name, submission_py, arg_strings)
    age = npy_file.split('/')[-1].split("_")[1]
    filename = f'/nas/longleaf/home/cctsai/proj_dip/slurm/{task_name}/age_{age}_.slurm'
    with open(filename, 'w') as script_file:
        script_file.write(script_content)

--file /work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP223008_233_T2w_SPC_NORM_5.npy --factor 2 --model_folder T2/KC-DIP --double_arm True--kspace_mse --kspace_boundary --kbound_outer_layer False
--file /work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP522586_342_T2w_SPC_NORM_5.npy --factor 2 --model_folder T2/KC-DIP --double_arm True--kspace_mse --kspace_boundary --kbound_outer_layer False
--file /work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP116056_205_T2w_SPC_NORM_9.npy --factor 2 --model_folder T2/KC-DIP --double_arm True--kspace_mse --kspace_boundary --kbound_outer_layer False
--file /work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP439083_146_T2w_SPC_NORM_5.npy --factor 2 --model_folder T2/KC-DIP --double_arm True--kspace_mse --kspace_boundary --kbound_outer_layer False
--file /work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP477045_317_T2w_SPC_NORM_5.npy --factor 2 --model_folder T2/KC-DIP --double_arm True--kspace_mse --kspace_boundary --kbound_outer_layer False
--file /work/users/c/c/cc

# Submit slurm files

In [6]:
slurm_files = []

# for tv_weight in [0.0005, 0.0001, 0.00005, 0.00001]:
#     exp_name = f"{exp_namespace}/tv_weight_{tv_weight}"


exp_name = f"{exp_namespace}"

# do not replace:
task_name = exp_name
#     task_name = f"{exp_name}/{int(factor*100)}"

for age in ages:
    slurm_files +=glob.glob(f"/nas/longleaf/home/cctsai/proj_dip/slurm/{task_name}/age_{age}_*.slurm")

print(len(slurm_files))
print(slurm_files)

10
['/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_233_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_342_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_205_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_146_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_317_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_92_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_17_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_54_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_109_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/KC-DIP/age_272_.slurm']


In [7]:
import subprocess

stdouts = []

for slurm_file in slurm_files:
    command = f'sbatch {slurm_file}'
    completed_process = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # Print the captured output
    print("Standard Output:", completed_process.stdout)
    stdouts.append(completed_process.stdout)
    
print("Total", len(stdouts), "jobs")

Standard Output: Submitted batch job 34745420

Standard Output: Submitted batch job 34745421

Standard Output: Submitted batch job 34745422

Standard Output: Submitted batch job 34745423

Standard Output: Submitted batch job 34745424

Standard Output: Submitted batch job 34745425

Standard Output: Submitted batch job 34745426

Standard Output: Submitted batch job 34745427

Standard Output: Submitted batch job 34745428

Standard Output: Submitted batch job 34745429

Total 10 jobs
